# Program 4 Assignment <br>Marianne Strauss
**Marianne Strauss<br>CSCI 3412 - Algorithms**

# Introduction
With the a list of characters, comics, and the appearance of each character in each comic, a social network is formed. This assignment is a program to determine the degree of seperation of an arbitrary character to Spider-Man. Data is retrieved from "porgat.txt" from the Social Characteristics of the Marvel Universe project. By reading in the data an Adjacency matrix can be built and based on this graph we can use various methods to determine the shortest path from the character to Spider-Man, or the Degree of Seperation.

In [1]:
using Printf
using SparseArrays # I dont know if matrix we use is sparse "enough" to fully use this matrix type
using LinearAlgebra


# Read Network

Using code from Marvel-Hint.jl (see citations). We first import the network built from list of comic characters, comics, and character appearances (edges) from porgat.txt. Data was compiled by and downloaded from Social Characteristics of the Marvel Universe project (also see citations).

In [2]:
"""
    read_network(pathname)

Read the Marvel universe network from the file. The format of the file is
given in the referenced papers.
"""
function read_network(pathname)
    # Reads the ith vertex from file
    function read_vertex(i, file)
	    s = readline(file)
        m = match(r"^([0-9]*)\s*\"(.*)\"$", s)
        if parse(Int, m[1]) != i
            error("Vertex number $i does not match expected number $line[1]")
        end
        return m[2]
    end
    # Process the input file
    open(pathname) do file
        # Read the *Vertices line
        parsed = split(readline(file))
        if parsed[1] != "*Vertices"
            error("Missing *Vertices line")
        end
        nvertices = parse(Int, parsed[2])
        ncharacters = parse(Int, parsed[3])
        ncomics = nvertices - ncharacters
        # Read vertices - characters and comics
        characters = [read_vertex(i, file) for i = 1:ncharacters]
        comics = [read_vertex(i, file) for i = ncharacters+1:nvertices]
        # Read *Edgeslist line
        if readline(file) != "*Edgeslist" then
            error("Missing *Edgeslist line")
        end
        # Read the edges - appearances
        appearances = zeros(Int, ncharacters, ncomics)
        while !eof(file)
            parsed = split(readline(file))
            character = parse(Int, parsed[1])
            for i = 2:length(parsed)
                comic = parse(Int, parsed[i]) - ncharacters
                appearances[character, comic] = 1
            end
        end
        return characters, comics, appearances
    end
end

read_network

In [3]:
# Read the network
println("Reading Marvel universe network")
characters, comics, appearances = read_network("porgat.txt")
ncharacters = length(characters)
ncomics = length(comics)
# Print some statistics
println("Number of characters = $ncharacters")
println("Number of comics = $ncomics")
nappearances = sum(appearances)
@printf("Mean books per character = %0.2f\n", nappearances / ncharacters)
@printf("Mean characters per book = %0.2f\n", nappearances / ncomics)
# Compute collaboration matrix
collaborations = appearances * appearances'
@show collaborations[1:30, 1:30]

Reading Marvel universe network
Number of characters = 6486
Number of comics = 12942
Mean books per character = 14.90
Mean characters per book = 7.47
collaborations[1:30, 1:30] = [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 9 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 9 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 45 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 

30×30 Matrix{Int64}:
 1  0  0  0  0  0  0  0  0   0  0  0  …  0   0  0  0   0  0  0  0   0  0  0
 0  9  0  0  0  0  0  0  0   0  0  0     0   0  0  0   0  0  0  0   0  0  0
 0  0  9  0  0  0  0  0  0   0  0  0     0   0  0  0   0  0  0  0   0  0  0
 0  0  0  3  0  0  0  0  0   0  0  0     0   0  0  0   0  0  0  0   0  0  0
 0  0  0  0  3  0  0  0  0   0  0  0     0   0  0  0   0  0  0  0   0  0  0
 0  0  0  0  0  4  0  0  0   0  0  0  …  0   0  0  0   0  0  0  0   0  0  0
 0  0  0  0  0  0  1  0  0   0  0  0     0   0  0  0   0  0  0  0   0  0  0
 0  0  0  0  0  0  0  2  0   0  0  0     0   0  0  0   0  0  0  0   0  0  0
 0  0  0  0  0  0  0  0  2   0  0  0     0   0  0  0   0  0  0  0   0  0  0
 0  0  0  0  0  0  0  0  0  45  0  0     0   1  0  0   0  0  0  0   0  0  0
 0  0  0  0  0  0  0  0  0   0  1  0  …  0   0  0  0   0  0  0  0   0  0  0
 0  0  0  0  0  0  0  0  0   0  0  3     0   0  0  0   0  0  0  0   0  0  0
 0  0  0  0  0  0  0  0  0   0  0  0     0   0  0  0   0  0  0  0  

Taking the appearance matrix returned from read_network() we can build a Adjacency matrix by multiplying the Appearance matrix by it's transpose. With this collaboration matrix we can count the walks of walks. With the colobaration matrix by caculating $A^n$, the lowest power $n$ where $(i,j)$ is non-zero is the length or distance from $i$ to $j$ (Definition taken from wiki article, see citations).

# Solution 1

With such a large square matrix (6486 x 6486), precomputing each power of $n$ (2 to 6) will be computationally expensive. Even with Julia's optimized BLAS functions expected runtime complexity is $Θ(n^3)$ due to the nature of multiplying matrices.

Based on hint given in class, the below method is an attempt to reduce runtime complexity to $O(n)$. Because we are only interested in one row (Spider-man's) and one arbitrary column, in theory we can discard the 2 "outer loops" and find the degree of seperation in constant time.

This should be done by just using dot multiplication on Spider-mans row with the arbitray column for each power. If $Collaboration[Spider-Man, arbirtary Character]$ or the result of that dot product of two vectors of spider-mans row and some arbitrary column is $>0$ than that power is the degree of seperation. So we just do the the dot product of Spidermans row and every column of Collaborations matrix and store it in rowvector vector. If $Collaboration[Spider-Man, arbirtary Character]$ is greater than zero (non-zero) we return that power.

In [36]:
function six_degrees_of_x(collabmatrix, xindex, characterindex)
    if xindex == characterindex
        return 0
    end
    
    # Initialize the row vector with the desired row of A
    rowvector = collabmatrix[xindex, :]
    
    # Initialize the result and power
    result = collabmatrix[xindex, characterindex]
    power = 1
    
    
    # Compute the powers of collabmatrix^n using row vector multiplication
    while result == 0 && power < 6
        for i in 1:size(rowvector, 1)
            rowvector[i] = dot(rowvector, collabmatrix[:, i])
        end
        power += 1
        result = rowvector[characterindex]
    end

    # Return the shortest path length between the characters
    if result == 0
        return nothing
    else
        return power
    end
end


six_degrees_of_x (generic function with 1 method)

In [40]:
test =  six_degrees_of_x(collaborations, 5306, 1)
println(test)

3


# Solution 2

This was more done because I couldnt get my Solution 1 to work and I need a relatively quick way to test my results. This method uses the LightGraphs package and its implmentation of the A* algorithm. This is useful because it can count the distance (number of walks) as an extension of Dijkstra's algorithm. The differance is that A* uses heuristics to guide it's search and does so from "a specified source to a specified goal". The drawback is that it has $O(b^d)$ space complexity. But for our uses it doesnt seem to be an issue and runs relatively quickly for each search.

In [6]:
using LightGraphs


g = SimpleGraph{Int64}(6486)
for i in 1:size(collaborations, 1)
    for j in 1:size(collaborations, 2)
        if collaborations[i,j] != 0
            add_edge!(g, i, j)
        end
    end
end

function shortestwithstar(a, b)
    
    d = length(a_star(g, a, b))
    return d
    
end

shortestwithstar (generic function with 1 method)

In [7]:
test = shortestwithstar(5306, 1)
println(test)

3


The below method also works and may have better performance of A*. 

In [8]:
test = gdistances(g, 5306)[1]
println(test)

3


# Solution 3

Below is the most bruteforce solution and would have a lot of overhead because I am just precomputing each $A^n$ and storing the resulting matrix in a vector. I will not run because it would theorectically take some time.

In [9]:
# #big calculation method
# tempmatrix = collaborations
# precomputed_vector = Matrix{Int64}[]

# for n in 1:6
#     push!(precomputed_vector, tempmatrix^n)
# end

# function bruteforce_six_degrees(v, characterindex)
#     for i in 1:6
#         if v[i][5306, characterindex] > 0
#             return i
#     end
#     return 0
# end

# Results

## Solution 1 Results

In [38]:
# Six degrees of Spider Man with Solution 1
# Test 1
listofcharactersindex = [1, 2, 3, 95, 389, 957, 1082, 1089, 1388, 1711, 
    1841, 2052, 2216, 2262, 2483, 2908, 3202, 3481, 3863, 4033, 
    4095, 4387, 4575, 4630, 4750, 5306, 5705, 6411, 6452, 6484, 64
]
println("Six degrees of Spider Man - Examples:")
for i in listofcharactersindex
    degrees = six_degrees_of_x(collaborations, 5306, i)
    
    if degrees == nothing
        println("$i " * characters[i] * " : >6")
    else
        println("$i " * characters[i] * " : $degrees")
    end
end

# Test 2
listofcharactersstr = ["ABSORBING MAN/CARL C", "ANTHROMORPHO", "AZOPARDI", "BEAUTIFUL DREAMER II", "BRAITHWAITE, JUDGE R", 
                        "CALLAHAN, DANNY", "CHAKRA II", "D'ANGELO, LIEUTENANT", "DARLEGUNG, GEN.", "GOOM", "HALE, JONAS", 
                        "HOFFMAN", "IRON GIANT MAN | MUT", "KATZENBERG, NICK", "LOGAN, WAYNE", "LUMP", "MANT/ERNEST",
                        "NEPTUNIUM", "O'HARA, ROBERT IRONG", "PET", "POTTS, VIRGINIA PEPP", "RANDAK", "SIKORSKY, RAYMOND", 
                        "SPIDER-MAN/PETER PAR", "TRUTH", "UNDERTOW", "VENOMM", "WILSON, JIM", "G'RATH"
]
println("More Six degrees of Spider Man Examples:")
for i in listofcharactersstr
    foundindex = findfirst(x -> occursin(i, x),  characters)
    degrees = six_degrees_of_x(collaborations, 5306, foundindex)
    
    if degrees == nothing
        println("$foundindex " * characters[foundindex] * " : >6")
    else
        println("$foundindex " * characters[foundindex] * " : $degrees")
    end
end

Six degrees of Spider Man - Examples:
1 24-HOUR MAN/EMMANUEL : 3
2 3-D MAN/CHARLES CHAN : 1
3 4-D MAN/MERCURIO : 2
95 AMAZO-MAXI-WOMAN/ : >6
389 BAV-TEK : 2
957 CERA, KAY : 1
1082 CLINTON, BILL : 1
1089 CLUMSY FOULUP : >6
1388 DE LA SPIROSA, COUNT : 2
1711 EL TORO ROJO : 2
1841 FENRIS : >6
2052 ÁGAMORA : 1
2216 GREASE : 2
2262 GROSS, IRA : 1
2483 HITLER, ADOLF : 2
2908 KUBIK : 2
3202 LITTLE, ABNER : 2
3481 MARTINEZ, ALITHA : 1
3863 MYLZOB'RYN, CHIEF : 2
4033 O'KEEFE, CHANDRA : 1
4095 OSWALD : >6
4387 PSI-BORG/ : 2
4575 REAGAN, NANCY : 2
4630 REPTYL : 1
4750 ROSE, MICHKA : 2
5306 SPIDER-MAN/PETER PAR : 0
5705 THANOS : 1
6411 ZANTOR : >6
6452 ZURI : 1
6484 STORMER : 2
64 AIRBORNE/ : 2
More Six degrees of Spider Man Examples:
16 ABSORBING MAN/CARL C : 1
157 ANTHROMORPHO : 2
304 AZOPARDI : 3
410 BEAUTIFUL DREAMER II : 2
688 BRAITHWAITE, JUDGE R : 2
835 CALLAHAN, DANNY : >6
982 CHAKRA II : 3
1306 D'ANGELO, LIEUTENANT : 1
1371 DARLEGUNG, GEN. : >6
2180 GOOM : 3
2309 HALE, JONAS : 1
2500 HOFF

# NOTE

very confused sometimes it returns 2 when 3 is expected (see 4750 ROSE, MICHKA etc) and sometimes returns 3 as expected (24-HOUR MAN/EMMANUEL). I am turning in as is because the solution below gives expected/correct results

## Solution 2 Results

In [24]:
# Six degrees of Spider Man with A*
# Test 1
listofcharactersindex = [1, 2, 3, 95, 389, 957, 1082, 1089, 1388, 1711, 
    1841, 2052, 2216, 2262, 2483, 2908, 3202, 3481, 3863, 4033, 
    4095, 4387, 4575, 4630, 4750, 5306, 5705, 6411, 6452, 6484, 64
]
println("Six degrees of Spider Man - Examples:")
for i in listofcharactersindex
    degrees = shortestwithstar(5306, i)
    
    if degrees == 0
        if i == 5306
                println("$i " * characters[i] * " : 0")
        else
                println("$i " * characters[i] * " : >6")
        end
    else
        println("$i " * characters[i] * " : $degrees")
    end
end

# Test 2
listofcharactersstr = ["ABSORBING MAN/CARL C", "ANTHROMORPHO", "AZOPARDI", "BEAUTIFUL DREAMER II", "BRAITHWAITE, JUDGE R", 
                        "CALLAHAN, DANNY", "CHAKRA II", "D'ANGELO, LIEUTENANT", "DARLEGUNG, GEN.", "GOOM", "HALE, JONAS", 
                        "HOFFMAN", "IRON GIANT MAN | MUT", "KATZENBERG, NICK", "LOGAN, WAYNE", "LUMP", "MANT/ERNEST",
                        "NEPTUNIUM", "O'HARA, ROBERT IRONG", "PET", "POTTS, VIRGINIA PEPP", "RANDAK", "SIKORSKY, RAYMOND", 
                        "SPIDER-MAN/PETER PAR", "TRUTH", "UNDERTOW", "VENOMM", "WILSON, JIM", "G'RATH"
]
println("More Six degrees of Spider Man Examples:")
for i in listofcharactersstr
    foundindex = findfirst(x -> occursin(i, x),  characters)
    degrees = shortestwithstar(5306, foundindex)
    
    if degrees == 0
        if foundindex == 5306
                println("$foundindex " * characters[foundindex] * " : 0")
        else
                println("$foundindex " * characters[foundindex] * " : >6")
        end
    else
        println("$foundindex " * characters[foundindex] * " : $degrees")
    end
end

Six degrees of Spider Man - Examples:
1 24-HOUR MAN/EMMANUEL : 3
2 3-D MAN/CHARLES CHAN : 1
3 4-D MAN/MERCURIO : 2
95 AMAZO-MAXI-WOMAN/ : >6
389 BAV-TEK : 3
957 CERA, KAY : 1
1082 CLINTON, BILL : 1
1089 CLUMSY FOULUP : >6
1388 DE LA SPIROSA, COUNT : 2
1711 EL TORO ROJO : 2
1841 FENRIS : >6
2052 ÁGAMORA : 1
2216 GREASE : 2
2262 GROSS, IRA : 1
2483 HITLER, ADOLF : 2
2908 KUBIK : 2
3202 LITTLE, ABNER : 2
3481 MARTINEZ, ALITHA : 1
3863 MYLZOB'RYN, CHIEF : 3
4033 O'KEEFE, CHANDRA : 1
4095 OSWALD : >6
4387 PSI-BORG/ : 2
4575 REAGAN, NANCY : 2
4630 REPTYL : 1
4750 ROSE, MICHKA : 3
5306 SPIDER-MAN/PETER PAR : 0
5705 THANOS : 1
6411 ZANTOR : >6
6452 ZURI : 1
6484 STORMER : 3
64 AIRBORNE/ : 2
More Six degrees of Spider Man Examples:
16 ABSORBING MAN/CARL C : 1
157 ANTHROMORPHO : 2
304 AZOPARDI : 3
410 BEAUTIFUL DREAMER II : 2
688 BRAITHWAITE, JUDGE R : 2
835 CALLAHAN, DANNY : >6
982 CHAKRA II : 3
1306 D'ANGELO, LIEUTENANT : 1
1371 DARLEGUNG, GEN. : >6
2180 GOOM : 3
2309 HALE, JONAS : 1
2500 HOFF

## Solution 3 Results

Not to be ran

In [12]:
# # Six degrees of Spider Man with the bruteforce method
# # Test 1
# listofcharactersindex = [1, 2, 3, 95, 389, 957, 1082, 1089, 1388, 1711, 
#     1841, 2052, 2216, 2262, 2483, 2908, 3202, 3481, 3863, 4033, 
#     4095, 4387, 4575, 4630, 4750, 5306, 5705, 6411, 6452, 6484, 64
# ]
# println("Six degrees of Spider Man - Examples:")
# for i in listofcharactersindex
#     degrees = bruteforce_six_degrees(precomputed_vector, i)
    
#     degrees == 0 && println("$i " * characters[i] * " : >6")
#     degrees < 6 && println("$i " * characters[i] * " : $degrees")
# end

# # Test 2
# listofcharactersstr = ["ABSORBING MAN/CARL C", "ANTHROMORPHO", "AZOPARDI", "BEAUTIFUL DREAMER II", "BRAITHWAITE, JUDGE R", 
#                         "CALLAHAN, DANNY", "CHAKRA II", "D'ANGELO, LIEUTENANT", "DARLEGUNG, GEN.", "GOOM", "HALE, JONAS", 
#                         "HOFFMAN", "IRON GIANT MAN | MUT", "KATZENBERG, NICK", "LOGAN, WAYNE", "LUMP", "MANT/ERNEST",
#                         "NEPTUNIUM", "O'HARA, ROBERT IRONG", "PET", "POTTS, VIRGINIA PEPP", "RANDAK", "SIKORSKY, RAYMOND", 
#                         "SPIDER-MAN/PETER PAR", "TRUTH", "UNDERTOW", "VENOMM", "WILSON, JIM", "G'RATH"
# ]
# println("More Six degrees of Spider Man Examples:")
# for i in listofcharactersstr
#     foundindex = findfirst(x -> occursin(i, x),  characters)
#     degrees = bruteforce_six_degrees(precomputed_vector, foundindex)
    
#     degrees == 0 && println("$foundindex " * characters[foundindex] * " : >6")
#     degrees < 6 && println("$foundindex " * characters[foundindex] * " : $degrees")
# end

## Citations

1. Williams, Doug. “Marvel-Hint.jl.” 
3. Williams, Doug. “Scooby Number.ipynb.” 
2. “Lightgraphs Documentation.” Path and Traversal · LightGraphs, https://docs.juliahub.com/LightGraphs/Xm08G/1.3.3/pathing/.
3. Miro, Joe, et al. “Social Characteristics of the Marvel Universe.” Dept. Matemàtiques i Informàtica, Universitat De Les Illes Balears, Feb. 2002, http://bioinfo.uib.es/~joemiro/marvel.html.
4. Wikipedia contributors. "Adjacency matrix." Wikipedia, The Free Encyclopedia. Wikipedia, The Free Encyclopedia, 11 Feb. 2023. Web. 5 May. 2023.
5. Wikipedia contributors. "A* search algorithm." Wikipedia, The Free Encyclopedia. Wikipedia, The Free Encyclopedia, 29 Apr. 2023. Web. 5 May. 2023.
6. “Dot Method.” Linear Algebra · The Julia Language, https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.dot-Tuple{Any,%20Any,%20Any}. 